Retrieval-Augmented Generation (RAG) Model for QA Bot

In [ ]:
!pip install PyPDF2 pinecone-client sentence-transformers cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import cohere
import numpy as np
from google.colab import userdata

pinecone = Pinecone(api_key=userdata.get('Pinecone'), environment='us-west1-gcp')  # update the api key with your own key
index_name = "qa-bot"

if not pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
index = pinecone.Index(index_name)

cohere_client = cohere.Client(api_key=userdata.get('Cohere'))# update the api key with your own key

embedding_model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def normalize_embedding(embedding):
    return embedding / np.linalg.norm(embedding)

def create_embeddings(text):
    embeddings = embedding_model.encode(text)
    normalized_embeddings = [normalize_embedding(emb) for emb in embeddings]
    return [embedding.tolist() for embedding in normalized_embeddings]

def store_embeddings_in_pinecone(text_chunks):
    embeddings = create_embeddings(text_chunks)
    vectors = [{"id": str(i), "values": embedding, "metadata": {"text": chunk}} for i, (embedding, chunk) in enumerate(zip(embeddings, text_chunks))]
    index.upsert(vectors=vectors, namespace="document-namespace")

def chunk_text(text, max_chunk_size=512):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def query_pinecone(query_text):
    query_embedding = normalize_embedding(embedding_model.encode([query_text])[0])
    query_results = index.query(vector=query_embedding.tolist(), top_k=5, include_metadata=True, namespace="document-namespace")
    return [match['metadata']['text'] for match in query_results['matches']]

def generate_answer_from_chunks(query, retrieved_chunks):
    combined_text = " ".join(retrieved_chunks)
    prompt = f"{combined_text}\n\nAnswer the following question: {query}"
    response = cohere_client.generate(
        model='c4ai-aya-23-35b',
        prompt=prompt,
        max_tokens=300
    )
    return response.generations[0].text

In [ ]:
# i have uploaded a pdf to the runnign section, please upload your pdf and change the pdf_path
pdf_path = "/content/VITA_Research_Paper.pdf"

reader = PdfReader(pdf_path)
document_text = ''
for page in reader.pages:
    document_text += page.extract_text()

text_chunks = chunk_text(document_text)
store_embeddings_in_pinecone(text_chunks)
print("Embeddings stored successfully!")

query = "what is VITA and how it is useful to students"
retrieved_chunks = query_pinecone(query)
generated_answer = generate_answer_from_chunks(query, retrieved_chunks)

print("\nRetrieved Document Sections:")
for chunk in retrieved_chunks:
    print(chunk)
    print("---")

print("\nGenerated Answer:")
print(generated_answer)

Embeddings stored successfully!

Retrieved Document Sections:
This shows that the platform is
particularly helpful in helping students grasp hard ideas
through targeted material delivery.
C.User Satisfaction
User satisfaction was measured by surveys that gathered
qualitative comments on the user experience, perceived effec-
tiveness, and overall happiness with the VITA platform.•Positive User Feedback: The majority of students (85%)
indicated a high level of satisfaction with the platform,
citing the simplicity of use, relevancy of individualized
content, and the participatory nature of the learning
experience as crucial factors.
•Perceived Effectiveness: 78% of students believed that
VITA made learning more interesting and effective com-
pared to traditional approaches.
---
This suggests that the individualized
learning strategy offered by VITA considerably boosted
student understanding and retention of material.
Fig. 5. Improvement in Test Scores (Pre-test vs. Post-test)
•Enhanced Le

APPROACH:

1. document processing: first we process the pdf file and extract the text using PyPDF2
2. text chunking: next we convert the extracted text into smaller chunks
3. text embeddigns: now we convert the text into vector embeddings using 'all-mpnet-base-v2' model
4. storing in pinecone: now we store the converted embeddings to pinecone database so that we access the relavent chunks based on the query.
5. querying: when someone asks a query or a question we convert the query into embeddings and ask the pinecone database for similar embeddings.
6. answer generation: now we take the relevant embeddings from the database and give it to 'c4ai-aya-23-35b' model which is available in cohere api then ask it to generate the answer for the question.